## 모델 불러오기 초기 설정

In [1]:
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import AutoTokenizer

from captum.attr import visualization
import torch

In [2]:
# Bert로 model 불러오기 

model = BertForSequenceClassification.from_pretrained('klue-bert-epoch-32').to("cuda:0")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [3]:
# encode a sentence
text_batch = ["야 너희 립스틱 이쁘다 엄마가 성적 3등 올랐다고 사준거야 나 오늘 남자친구랑 데이트 하는데 좀 빌려줘라 안 돼 저번에도 너 빌려 가서 망가트렸잖아 너 많이 컸다 이제 말대꾸도 하냐 그게 아니라 엄마가 백화점 가서 특별히 사 준 건데 그러니까 더 좋네 남자친구한테 잘 보이고 오늘 내가 빌려 갈테니까 그런 줄 알아 어디 가서 꼬바르지 말고 알아서 잘 해라 안 되는데"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda:0")
attention_mask = encoding['attention_mask'].to("cuda:0")

In [4]:
# initialize the explanations generator
explanations = Generator(model)

## generate_LRP 코드 이해하기 (ExplanationGenerator.py 안)

### one hot vector와 model.relprop의 cam을 이해하기

In [18]:
import numpy as np 

output = model(input_ids=input_ids, attention_mask=attention_mask)[0]
index = np.argmax(output.cpu().data.numpy(), axis=-1)
one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
one_hot[0, index] = 1
one_hot_vector = one_hot
print(one_hot_vector)

[[0. 0. 1. 0. 0.]]


In [19]:
kwargs = {"alpha": 1}
cam = model.relprop(torch.tensor(one_hot_vector).to(input_ids.device), **kwargs)
print(cam)

conservation:  tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)
tensor([[[ 6.6502e-06,  7.0124e-06,  6.5108e-06,  ...,  1.0495e-05,
           1.2351e-05,  1.1076e-06],
         [-6.0508e-08, -1.6266e-07,  1.4527e-05,  ...,  6.3141e-06,
           2.7440e-06,  8.4078e-06],
         [ 8.8628e-06,  1.2531e-05,  7.4550e-06,  ...,  1.0610e-05,
           1.8788e-05,  2.2743e-05],
         ...,
         [-2.1188e-05,  4.8777e-05,  4.3548e-05,  ..., -3.2191e-05,
           1.4737e-05, -3.9474e-05],
         [ 8.6562e-07,  4.0281e-07,  3.0491e-06,  ...,  4.2468e-06,
           8.0335e-06, -1.6084e-09],
         [ 2.4430e-06, -3.4200e-08, -5.6509e-06,  ...,  2.0533e-07,
          -2.6913e-06,  5.1070e-06]]], device='cuda:0', grad_fn=<MulBackward0>)


In [20]:
print(cam.shape) ## 103은 Input ids 길이

torch.Size([1, 103, 768])


In [21]:
print(input_ids)

tensor([[    2,  1396,  6928, 16960, 13854,  2062,  4122,  2116,  4610,    23,
          2491,  4990,  4683,  1233,  2456,  2180,  2275,   717,  3822,  3997,
          2720,  2251,  2379,  8698,  1889, 13964,  1556,  7004,  2810,  2181,
          1378,   857, 14247,  6509,   743,  7004,   543,  2112, 13731,  2265,
          2542,  9958,   743,  3732,  1718,  2062,  3699,  1041,  2104,  2660,
          2119,  1889,  2529,  4603,  3614,  2181,  4122,  2116,  5269,   543,
          2112,  7761,  1233,  1566,  6461,  4719,   831,  1560,  2203,  3997,
          2720,  2251,  2470,  2201,  1521,  3783,  2088,  3822,   732,  2116,
          7004,   547,  2201,  3707,  3637,  1567,  4860,  4069,   543,  2112,
           676, 23746,  2118,  1041,  2088,  4860,  2112,  1521,  8982,  1378,
           859, 13964,     3]], device='cuda:0')


In [12]:
print(input_ids.shape)

torch.Size([1, 103])


### model.relprop 상세

In [7]:
print(model.bert.encoder.layer)

ModuleList(
  (0): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (matmul1): MatMul()
        (matmul2): MatMul()
        (softmax): Softmax(dim=-1)
        (add): Add()
        (mul): Mul()
        (clone): Clone()
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (add): Add()
      )
      (clone): Clone()
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELU()
    )
    (output): BertOutput(
      (dense): Linear(i

In [22]:
one_hot = torch.from_numpy(one_hot).requires_grad_(True)
one_hot = torch.sum(one_hot.cuda() * output)
model.zero_grad()
one_hot.backward(retain_graph=True)
model.relprop(torch.tensor(one_hot_vector).to(input_ids.device), **kwargs)

conservation:  tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)


tensor([[[ 6.6502e-06,  7.0124e-06,  6.5108e-06,  ...,  1.0495e-05,
           1.2351e-05,  1.1076e-06],
         [-6.0508e-08, -1.6266e-07,  1.4527e-05,  ...,  6.3141e-06,
           2.7440e-06,  8.4078e-06],
         [ 8.8628e-06,  1.2531e-05,  7.4550e-06,  ...,  1.0610e-05,
           1.8788e-05,  2.2743e-05],
         ...,
         [-2.1188e-05,  4.8777e-05,  4.3548e-05,  ..., -3.2191e-05,
           1.4737e-05, -3.9474e-05],
         [ 8.6562e-07,  4.0281e-07,  3.0491e-06,  ...,  4.2468e-06,
           8.0335e-06, -1.6084e-09],
         [ 2.4430e-06, -3.4200e-08, -5.6509e-06,  ...,  2.0533e-07,
          -2.6913e-06,  5.1070e-06]]], device='cuda:0', grad_fn=<MulBackward0>)

### attention gradients와 cam을 사용하고, rollout하기

In [33]:
blocks = model.bert.encoder.layer
cams = []
for blk in blocks:
    grad = blk.attention.self.get_attn_gradients()
    cam = blk.attention.self.get_attn_cam()
    cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
    grad = grad[0].reshape(-1, grad.shape[-1], grad.shape[-1])
    cam = grad * cam
    cam = cam.clamp(min=0).mean(dim=0)
    cams.append(cam.unsqueeze(0))
    print(cam.unsqueeze(0).shape)

print(cams)


torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
torch.Size([1, 103, 103])
[tensor([[[1.5734e-10, 2.1191e-11, 2.0456e-11,  ..., 1.2488e-11,
          3.1215e-11, 7.1514e-11],
         [1.2349e-10, 3.1889e-10, 6.6744e-10,  ..., 1.0621e-11,
          1.5431e-11, 1.3715e-11],
         [2.0703e-11, 7.2510e-11, 1.0170e-10,  ..., 2.1040e-11,
          1.6521e-10, 1.4448e-11],
         ...,
         [1.3246e-09, 5.4441e-09, 2.7564e-09,  ..., 7.1101e-10,
          1.2557e-08, 1.8499e-08],
         [4.1150e-12, 5.5394e-12, 2.9210e-11,  ..., 3.1085e-11,
          1.0869e-11, 3.0344e-12],
         [4.1889e-10, 1.6871e-10, 1.5483e-10,  ..., 5.0136e-10,
          1.5928e-10, 5.3511e-10]]], device='cuda:0',
       grad_fn=<UnsqueezeBackward0>), tensor([[[2.4835e-10, 

In [34]:
def compute_rollout_attention(all_layer_matrices, start_layer=0):
    # adding residual consideration- code adapted from https://github.com/samiraabnar/attention_flow
    num_tokens = all_layer_matrices[0].shape[1]
    batch_size = all_layer_matrices[0].shape[0]
    eye = torch.eye(num_tokens).expand(batch_size, num_tokens, num_tokens).to(all_layer_matrices[0].device)
    all_layer_matrices = [all_layer_matrices[i] + eye for i in range(len(all_layer_matrices))]
    matrices_aug = [all_layer_matrices[i] / all_layer_matrices[i].sum(dim=-1, keepdim=True)
                          for i in range(len(all_layer_matrices))]
    joint_attention = matrices_aug[start_layer]
    for i in range(start_layer+1, len(matrices_aug)):
        joint_attention = matrices_aug[i].bmm(joint_attention)
    return joint_attention

In [37]:
rollout = compute_rollout_attention(cams, start_layer=0)
print(rollout)
print(rollout.shape)

tensor([[[9.9954e-01, 5.3428e-06, 1.8052e-06,  ..., 1.3060e-05,
          1.2558e-05, 2.8695e-07],
         [7.6200e-09, 1.0000e+00, 6.9643e-08,  ..., 9.2456e-10,
          1.4790e-09, 9.9302e-09],
         [1.1842e-09, 3.7269e-09, 1.0000e+00,  ..., 1.2881e-10,
          2.8959e-10, 9.8326e-10],
         ...,
         [5.3553e-09, 5.9267e-09, 4.0076e-09,  ..., 1.0000e+00,
          4.5092e-08, 1.9572e-08],
         [5.2646e-09, 1.5732e-09, 7.9207e-10,  ..., 7.3890e-09,
          1.0000e+00, 8.9342e-09],
         [2.5423e-07, 3.0861e-08, 2.3467e-08,  ..., 3.1751e-08,
          6.2350e-08, 1.0000e+00]]], device='cuda:0', grad_fn=<BmmBackward0>)
torch.Size([1, 103, 103])


In [41]:
print(rollout[:, 0].min())

tensor(4.5220e-08, device='cuda:0', grad_fn=<MinBackward1>)


In [42]:
print(rollout[:, 0, 0])

tensor([0.9995], device='cuda:0', grad_fn=<SelectBackward>)


In [44]:
print(rollout[:, 0])
print(rollout[:, 0].shape)

tensor([[9.9954e-01, 5.3428e-06, 1.8052e-06, 1.6658e-06, 6.0884e-07, 3.2363e-07,
         1.3004e-06, 5.8906e-08, 1.8044e-07, 7.3464e-08, 9.0848e-08, 1.0320e-07,
         2.3398e-07, 3.0671e-06, 1.1857e-06, 5.0555e-06, 8.3092e-06, 3.1887e-07,
         2.0381e-07, 7.1675e-08, 1.9397e-07, 4.5220e-08, 2.0571e-07, 7.4925e-07,
         8.1097e-07, 1.5228e-07, 2.8827e-05, 3.4590e-05, 1.5360e-05, 1.2209e-05,
         1.4661e-05, 1.4500e-05, 2.5128e-06, 7.7164e-07, 6.0322e-06, 1.1441e-05,
         3.0851e-06, 1.5179e-06, 7.2159e-06, 2.3456e-06, 6.2100e-07, 1.2983e-06,
         2.2889e-07, 8.7674e-08, 2.4717e-06, 1.1569e-07, 3.0736e-07, 6.1155e-07,
         3.4376e-07, 7.6198e-07, 7.2977e-08, 4.6611e-07, 1.2529e-06, 9.4217e-06,
         8.7931e-06, 2.4748e-06, 2.4383e-06, 7.4513e-07, 1.5902e-06, 2.9190e-07,
         2.1643e-07, 1.1251e-06, 8.0224e-06, 4.5463e-06, 7.4656e-06, 1.4571e-05,
         6.2419e-06, 2.9069e-06, 8.6139e-06, 1.3326e-06, 4.0356e-06, 3.4298e-07,
         9.4826e-08, 1.0640e

## expl 이해하기

In [45]:
# encode a sentence
text_batch = ["야 너희 립스틱 이쁘다 엄마가 성적 3등 올랐다고 사준거야 나 오늘 남자친구랑 데이트 하는데 좀 빌려줘라 안 돼 저번에도 너 빌려 가서 망가트렸잖아 너 많이 컸다 이제 말대꾸도 하냐 그게 아니라 엄마가 백화점 가서 특별히 사 준 건데 그러니까 더 좋네 남자친구한테 잘 보이고 오늘 내가 빌려 갈테니까 그런 줄 알아 어디 가서 꼬바르지 말고 알아서 잘 해라 안 되는데"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda:0")
attention_mask = encoding['attention_mask'].to("cuda:0")

# initialize the explanations generator
explanations = Generator(model)

classifications = ["020121","000001","02051","020811","020819"]

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]

conservation:  tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)


In [47]:
print(expl)

tensor([4.5220e-08, 5.3428e-06, 1.8052e-06, 1.6658e-06, 6.0884e-07, 3.2363e-07,
        1.3004e-06, 5.8906e-08, 1.8044e-07, 7.3464e-08, 9.0848e-08, 1.0320e-07,
        2.3398e-07, 3.0671e-06, 1.1857e-06, 5.0555e-06, 8.3092e-06, 3.1887e-07,
        2.0381e-07, 7.1675e-08, 1.9397e-07, 4.5220e-08, 2.0571e-07, 7.4925e-07,
        8.1097e-07, 1.5228e-07, 2.8827e-05, 3.4590e-05, 1.5360e-05, 1.2209e-05,
        1.4661e-05, 1.4500e-05, 2.5128e-06, 7.7164e-07, 6.0322e-06, 1.1441e-05,
        3.0851e-06, 1.5179e-06, 7.2159e-06, 2.3456e-06, 6.2100e-07, 1.2983e-06,
        2.2889e-07, 8.7674e-08, 2.4717e-06, 1.1569e-07, 3.0736e-07, 6.1155e-07,
        3.4376e-07, 7.6198e-07, 7.2977e-08, 4.6611e-07, 1.2529e-06, 9.4217e-06,
        8.7931e-06, 2.4748e-06, 2.4383e-06, 7.4513e-07, 1.5902e-06, 2.9190e-07,
        2.1643e-07, 1.1251e-06, 8.0224e-06, 4.5463e-06, 7.4656e-06, 1.4571e-05,
        6.2419e-06, 2.9069e-06, 8.6139e-06, 1.3326e-06, 4.0356e-06, 3.4298e-07,
        9.4826e-08, 1.0640e-07, 1.7212e-

In [48]:
expl.shape

torch.Size([103])

In [49]:
expl = (expl - expl.min()) / (expl.max() - expl.min()) ## min-max scaling

In [50]:
print(expl)

tensor([0.0000e+00, 1.5335e-01, 5.0949e-02, 4.6912e-02, 1.6316e-02, 8.0594e-03,
        3.6336e-02, 3.9620e-04, 3.9145e-03, 8.1762e-04, 1.3208e-03, 1.6785e-03,
        5.4643e-03, 8.7476e-02, 3.3014e-02, 1.4504e-01, 2.3922e-01, 7.9217e-03,
        4.5907e-03, 7.6582e-04, 4.3060e-03, 0.0000e+00, 4.6458e-03, 2.0380e-02,
        2.2167e-02, 3.0993e-03, 8.3317e-01, 1.0000e+00, 4.4333e-01, 3.5211e-01,
        4.2309e-01, 4.1843e-01, 7.1430e-02, 2.1028e-02, 1.7331e-01, 3.2988e-01,
        8.7997e-02, 4.2632e-02, 2.0757e-01, 6.6590e-02, 1.6668e-02, 3.6273e-02,
        5.3169e-03, 1.2290e-03, 7.0240e-02, 2.0399e-03, 7.5882e-03, 1.6394e-02,
        8.6422e-03, 2.0749e-02, 8.0351e-04, 1.2184e-02, 3.4959e-02, 2.7143e-01,
        2.5323e-01, 7.0332e-02, 6.9274e-02, 2.0261e-02, 4.4724e-02, 7.1407e-03,
        4.9561e-03, 3.1260e-02, 2.3092e-01, 1.3030e-01, 2.1480e-01, 4.2049e-01,
        1.7938e-01, 8.2840e-02, 2.4804e-01, 3.7266e-02, 1.1551e-01, 8.6196e-03,
        1.4360e-03, 1.7709e-03, 4.8517e-

In [53]:
sum(expl.cpu().data.numpy())

13.219114140083548